In [2]:
import os

# Set the PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT environment variable to 0.6 seconds
# os.environ['PYDEVD_WARN_SLOW_RESOLVE_TIMEOUT'] = '0.7'

import torch
import matplotlib.pyplot as plt
import networkx as nx
import collections

from copy import deepcopy

# local
import optimization_utils as ou

import sys
if '..' not in sys.path:
    sys.path.insert(0, '..')


from datasets.import_dataset import import_dataset
from trainer import Trainer
from utils.plotting import *
from utils import utils
from utils import utils_pyg as up
import datasets.simulations as sim
import link_prediction as lp





device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device = {device}')


%load_ext autoreload
%autoreload 2

device = cuda


In [5]:
'''run big and ie on a chosen dataset to find the optimal number of communities and number of iterations. For each experiment choose a percentage '''

model_name = 'piegam'
ds_name = 'texas'
s_regs = [0.0]
l1_regs = [1.0]
dim_feats = [22]
n_iters_feats = [2000]
lr_feats = [0.000004]
first_funcs_in_fit = ['fit_prior']
n_iters_prior = [2000]
lr_prior = [0.000005]
noise_amps = [0.05]
n_back_forth = [7]
global_config_base = True
densify = False
attr_opt = False
test_p = 0.1
val_p = 0.0
n_reps = 3

# ========= RESULTS FOLDERS =========
if not torch.cuda.is_available():
    raise Exception('CUDA not available')
device = torch.device('cuda')
printd(f'Using device: {device}')


range_triplets = [
    ['clamiter_init','s_reg', s_regs],
    ['clamiter_init','l1_reg', l1_regs],
    ['clamiter_init', 'dim_feat', dim_feats],
    ['feat_opt','n_iter', n_iters_feats],
    ['feat_opt','lr', lr_feats],
]

if model_name in ['pclam', 'piegam']:
    range_triplets += [
        # ['clamiter_init','dim_attr', args.dim_attr],
        ['back_forth', 'first_func_in_fit', first_funcs_in_fit],
        ['prior_opt','n_iter', n_iters_prior],
        ['prior_opt','lr', lr_prior],
        ['prior_opt','noise_amp', noise_amps],
        ['back_forth','n_back_forth', n_back_forth]
    ]
# Create the file if it doesn't exist
#todo: test the datasets: photo, texas, facebook, squirrel and crocodile
ou.cross_val_link(
    ds_name=ds_name,
    model_name=model_name,
    range_triplets=range_triplets,
    global_config_base=global_config_base,
    densify=densify,
    attr_opt=attr_opt,
    test_p=test_p,
    val_p=val_p,
    n_reps=n_reps,
    device=device
    )






/tmp/ipykernel_23888/614966897.py:26:::  Using device: cuda


/home/user/Documents/danny/AAAI_pieclam/experiments/../trainer.py:316:::  
 2024-12-11 02:26:11 starting optimization of piegam on texas on device cuda

 configs_dict: 
{
    "clamiter_init": {
        "dim_feat": 22,
        "dim_attr": 100,
        "s_reg": 0.0,
        "l1_reg": 1.0,
        "T": 1,
        "hidden_dim": 64,
        "num_coupling_blocks": 32,
        "num_layers_mlp": 2
    },
    "feat_opt": {
        "lr": 4e-06,
        "n_iter": 2000,
        "early_stop": 0
    },
    "prior_opt": {
        "n_iter": 2000,
        "lr": 5e-06,
        "noise_amp": 0.05,
        "weight_decay": 0.1,
        "early_stop": 0
    },
    "back_forth": {
        "n_back_forth": 7,
        "scheduler_step_size": 3,
        "scheduler_gamma": 0.5,
        "early_stop_fit": 0,
        "first_func_in_fit": "fit_prior"
    }
}


/home/user/Documents/danny/AAAI_pieclam/experiments/../clamiter.py:543:::  
in fit,
first_func_in_